In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
display(spark.sql('SHOW CATALOGS'))
display(spark.sql('SHOW SCHEMAS in john_snow_labs_acute_hospital_activity_and_nhs_beds_information'))

spark.sql("USE CATALOG dev")
spark.sql("USE SCHEMA bronze")

In [0]:
## helper funcitons

#read tables
def read_table(table_name):
  return spark.read.table(table_name)

def add_metadata(df):
    df =(
        df.withColumn('ingested_at', current_timestamp())
        .withColumn('source', lit('dbMarketplace'))
    )
    return df

def save_table(df, table_name):
    df.write.mode('append').saveAsTable(table_name)
    print(f'Saved table {table_name}')

    


In [0]:
tables = spark.sql('SHOW TABLES in john_snow_labs_acute_hospital_activity_and_nhs_beds_information.acute_hospital_activity_and_nhs_beds_information')
tableNames = [row['tableName'] for row in tables.collect()]

SOURCE = 'john_snow_labs_acute_hospital_activity_and_nhs_beds_information.acute_hospital_activity_and_nhs_beds_information'
TARGET = 'ara_db_sbox_001_useast.bronze'

spark.sql(f'CREATE SCHEMA IF NOT EXISTS {TARGET}')
for tn in tableNames:
    src = f'{SOURCE}.{tn}'
    df = read_table(src)
    df = add_metadata(df)
    print(f'Reading table {src}')
    save_table(df, f'{TARGET}.{tn}')

  